In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import moscot
from anndata import AnnData
import numpy as np
from moscot.backends.ott import FGWSolver, SinkhornSolver, GWSolver, LRSinkhornSolver
from moscot.solvers._data import TaggedArray, Tag
import matplotlib.pyplot as plt
import jax.numpy as jnp
from typing import *
import pandas as pd
import networkx as nx
import jax.numpy as jnp
from moscot.problems.time._lineage import TemporalProblem
import os
import wot

In [3]:
import ott

In [4]:
data_path = "/Users/DominikKlein/Documents/PhD/data/wot"


VAR_GENE_DS_PATH = os.path.join(data_path, 'ExprMatrix.var.genes.h5ad')
CELL_DAYS_PATH = os.path.join(data_path, 'cell_days.txt')
SERUM_CELL_IDS_PATH = os.path.join(data_path, 'serum_cell_ids.txt')
CELL_GROWTH_PATH = os.path.join(data_path, 'growth_gs_init.txt')

In [5]:
adata = wot.io.read_dataset(VAR_GENE_DS_PATH, obs=[CELL_DAYS_PATH, CELL_GROWTH_PATH], obs_filter=SERUM_CELL_IDS_PATH)
adata.shape

(175472, 1479)

In [15]:
adata = adata[adata.obs.day.isin([11, 11.5])].copy()
adata.X = adata.X.todense()

## Run WOT

In [16]:
ot_model = wot.ot.OTModel(adata,epsilon = 0.05, lambda1 = 1,lambda2 = 50) 

In [17]:
tmap_annotated = ot_model.compute_transport_map(11,11.5)

## Run Moscot

In [18]:
tp = TemporalProblem(adata, solver=SinkhornSolver(jit=False))

In [19]:
adata.obs.day = adata.obs.day.astype('category')

In [20]:
tp.prepare(key="day", subset=[11, 11.5])

In [21]:
tp.solve(eps=0.05, tau_a=1, tau_b=50)

## Compare

In [26]:
tp.transport_matrix

AttributeError: 'float' object has no attribute 'transport_matrix'

In [30]:
tp._solutions[(11.0, 11.5)].solution.transport_matrix.shape

KeyboardInterrupt: 

In [33]:
ot_model.matrix

AnnData object with n_obs × n_vars = 4402 × 1479
    obs: 'day', 'cell_growth_rate'

In [34]:
adata

AnnData object with n_obs × n_vars = 4402 × 1479
    obs: 'day', 'cell_growth_rate'